In [26]:
import pyterrier as pt
from pathlib import Path
import json

if not pt.started():
    pt.init(tqdm="notebook")
    # pt.init()



dataset = pt.get_dataset('irds:trec-fair/2022/train')
indexer = pt.IterDictIndexer('C:/Users/WalterKahn/Documents/F - Overig/InfoRetrieval/Git/ir-group-30/trec-fair_2022_multi', 
                             meta={"docno" : 20, "gender": 200, "title":512, "gender_category":100, "text":10000}, meta_reverse = ['docno'])



In [ ]:
index_ref = indexer.index(dataset.get_corpus_iter(), fields=['title', 'text', 'url'])

In [4]:
index_ref = pt.IndexRef.of("C:/Users/WalterKahn/Documents/F - Overig/InfoRetrieval/Git/ir-group-30/trec-fair_2022_multi")

In [5]:
import pandas as pd
import ast


def re_rank_by_gender(raw_df):
    df = raw_df.copy()
    df['gender'] = df['gender'].apply(lambda x: x.strip('[]').split(','))
    df['gender'] = df['gender'].apply(lambda x: x[0] if x else 'unknown')
    
    ranked_df = pd.DataFrame(columns=df.columns)
    
    desired_distribution = {'Man':0.3, 'Woman':0.3, 'Non-binary':0.3, 'Unknown':0.1, 'null':0.01}
    counts = {val: 0 for val in desired_distribution}
    
    while not df.empty:
        current_dist = {val: counts[val] / sum(counts.values()) if sum(counts.values()) > 0 else 0 for val in counts}
        next_att = max(desired_distribution.keys(), key=lambda x: desired_distribution[x] - current_dist.get(x, 0))
        
        if not df[df['gender_category'] == next_att].empty:
            next_doc = df[df['gender_category'] == next_att].sort_values(by='score', ascending=False).head(1)
            df = df.drop(next_doc.index)
            counts[next_att] += 1
            ranked_df = pd.concat([ranked_df, next_doc], ignore_index=True)
        else:
            # Remove the exhausted category and redistribute its target distribution
            del desired_distribution[next_att]
            if desired_distribution:  # Ensure there are still categories remaining
                total_distribution = sum(desired_distribution.values())
                for key in desired_distribution.keys():
                    desired_distribution[key] /= total_distribution
            else:
                break  # All categories are exhausted

    initial_score = 10
    score_decrement = 0.01
    ranked_df['score'] = initial_score - (ranked_df.index * score_decrement)
    ranked_df['rank'] = ranked_df.index
    
    return ranked_df

In [45]:
datamodel = pt.BatchRetrieve(index_ref, wmodel="BM25",metadata=["gender","gender_category","title","text"])

16:10:27.299 [main] WARN org.terrier.structures.FSADocumentIndex - This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based models such as BM25F, change to index.document.class in the index  properties file to FSAFieldDocumentIndex or FSADocumentIndexInMemFields to support efficient retrieval. If you don't use (e.g.) BM25F, this warning can be ignored
16:10:28.484 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 7,3 GiB of memory would be required.


In [42]:
with open("eval_topics.jsonl","r") as queries:
    data = json.load(queries)

queries = []

# while (i < 45):
    
   
len(data["keywords"]["0"])

for i in range(0, 46):
    queries += data["keywords"][str(i)]

len(queries)

2336

In [ ]:
def AWRF(query, datamodel, beta=0.5):
    """
    Calculate the Average Weighted Relevance Feedback (AWRF) score for a given query.

    Parameters:
        query (str): The query for which to calculate the AWRF score.
        datamodel: The data model or retrieval system used to search for documents.
        beta (float): Weighting factor between 0 and 1.

    Returns:
        float: The AWRF score for the query.
    """
    # Retrieve documents for the query using the data model
    search_results = datamodel.search(query)

    # Extract relevance feedback scores for the retrieved documents
    relevance_feedback = search_results["score"]

    # Original relevance scores assigned by the retrieval system
    original_scores = search_results["original_score"]  # Assuming this information is available

    # Calculate AWRF score for each document
    awrf_scores = []
    for rf, original in zip(relevance_feedback, original_scores):
        awrf_score = beta * rf + (1 - beta) * original
        awrf_scores.append(awrf_score)

    # Calculate the average AWRF score
    avg_awrf_score = sum(awrf_scores) / len(awrf_scores)

    return avg_awrf_score

In [47]:
datamodel.search("human")["score"]
    

0      8.944450
1      8.910652
2      8.908404
3      8.895001
4      8.874357
         ...   
995    8.409047
996    8.408914
997    8.408914
998    8.408715
999    8.408639
Name: score, Length: 1000, dtype: float64

In [ ]:
def AWRF(query,datamodel):
    
    search_results =  datamodel.search(query)["score"]
    
    relevance_feedback = search_results["score"]
    

In [6]:
pipeline = pt.BatchRetrieve(index_ref, wmodel='BM25', metadata=['gender', 'gender_category']) >> pt.apply.generic(re_rank_by_gender)
pipeline.search('scientist biography')

13:58:00.028 [main] WARN org.terrier.structures.FSADocumentIndex - This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based models such as BM25F, change to index.document.class in the index  properties file to FSAFieldDocumentIndex or FSADocumentIndexInMemFields to support efficient retrieval. If you don't use (e.g.) BM25F, this warning can be ignored
13:58:01.521 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 7,3 GiB of memory would be required.


C:\Users\WalterKahn\AppData\Local\Temp\ipykernel_35640\478842421.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ranked_df = pd.concat([ranked_df, next_doc], ignore_index=True)


,qid,docid,gender,gender_category,rank,score,query
0,1,5194694,"""male""",Man,0,10.00,scientist biography
1,1,1001375,"""female""",Woman,1,9.99,scientist biography
2,1,1586724,,Unknown,2,9.98,scientist biography
3,1,5908501,"""male""",Man,3,9.97,scientist biography
4,1,4638839,"""female""",Woman,4,9.96,scientist biography
...,...,...,...,...,...,...,...
995,1,4355895,"""male""",Man,995,0.05,scientist biography
996,1,5013317,"""male""",Man,996,0.04,scientist biography
997,1,5027514,"""male""",Man,997,0.03,scientist biography
998,1,895225,"""male""",Man,998,0.02,scientist biography
